In [1]:
from IPython.display import Image
import ee
import ee.mapclient

ee.Initialize()
collection_precipitation = ee.ImageCollection("projects/fao-wapor/L1_PCP5K");

meanPrecipitation2015 = collection_precipitation.reduce(ee.Reducer.mean());
sdPrecipitation2015 = collection_precipitation.reduce(ee.Reducer.stdDev());
sdPrecipitation2015.getInfo()

{u'bands': [{u'crs': u'EPSG:4326',
   u'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
   u'data_type': {u'precision': u'double', u'type': u'PixelType'},
   u'id': u'b1_stdDev'}],
 u'type': u'Image'}

In [2]:
region = [[-25.0, -37.0], [60.0, -41.0], [58.0, 39.0], [-31.0, 38.0], [-25.0, -37.0]]
url_sd = sdPrecipitation2015.getThumbUrl({
    'bands' : 'b1_stdDev',
    'palette':'faffb8,79e6ff,6daeff,346abc,fc6fff,b032ff,9913ff',
    'min': -8,
    'max' : 12,
    'region':region
})

In [4]:
%timeit Image(url=url_sd)

10000 loops, best of 3: 33.1 µs per loop


In [5]:
Image(url=url_sd)

In [6]:
L1_AGBPSeasonals = ee.ImageCollection("projects/fao-wapor/L1_AGBP")

def maskNoData(image):
    return image.updateMask(image.gte(0))

#Above Ground Biomass Production with masked NoData (pixel < 0)
L1_AGBPSeasonalMasked =L1_AGBPSeasonals.map(maskNoData)
L1_AGBPSummedYearly = L1_AGBPSeasonalMasked.sum();

L1_AGBPSummedYearly.getInfo()
url_summedYearly = L1_AGBPSummedYearly.getThumbUrl({
    'bands' : 'b1',
    'palette':'ffffff,0f0f0f,f0f0f0',    
    'max' : 750,
    'region':region
})
Image(url=url_summedYearly)

In [8]:
%prun -s cumtime L1_AGBPSummedYearly = L1_AGBPSeasonalMasked.sum()

In [27]:
fileName = 'maskerato'
downConfig = {'region': region};

export = ee.batch.Export.image(L1_AGBPSummedYearly, fileName,downConfig)
#export.start();